# Postprocess SWAT Simulations (5) - Fancy Interactive Map

In this notebook, we'll extend the tutorial (4) to pruduce a fancy **interactive map** of water yield changes in each season using [Folium](http://python-visualization.github.io/folium/). If you already went through the tutorial (4), you can directly jump to the last part of this notebook to see the interactive map. Folium provides many options to customize these maps to make them more interactive, attractive. We'll use popup text to show more information. Moreover, the map can be directly inserted into webpages. 

[Folium](http://python-visualization.github.io/folium/)builds on the data wrangling strengths of the Python ecosystem and the mapping strengths of the Leaflet.js library. That is to say, manipulate data in Python, then visualize it in on a Leaflet map via Folium. The library has a number of built-in tilesets from **OpenStreetMap**, **Mapbox**, and **Stamen**. Folium supports both GeoJSON and TopoJSON overlays, as well as the binding of data to those overlays to create choropleth maps with color-brewer color schemes.

*It is worth noting that all data in this series are fake data and only are used to show how to postprocess SWAT simulations through open source tools.*

## 1. Load all needed libraries

In [1]:
import geopandas as gpd 
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import folium

%matplotlib inline

## 2. Read Watershed map

Here the operations are similar to the [tutorial 3](https://www.linkedin.com/pulse/postprocess-swat-simulations-3-view-watershed-geopandas-yin/). *Read shapefile, transform projection, and add a column of coordinate*.

In [2]:
geo_df = gpd.read_file("data\Watersheds\subs1.shp")
geo_df.set_index(geo_df["Subbasin"].astype(int), inplace=True)
geo_df_ll = geo_df.to_crs(epsg=4269)

geo_df_ll['coords'] = geo_df_ll['geometry'].apply(lambda x: x.representative_point().coords[:])
geo_df_ll['coords'] = [coords[0] for coords in geo_df_ll['coords']]

## 3. Calculate Seasonal changes in each subbasin
The calculation and statistical analysis are similar to the [tutorial 2](https://www.linkedin.com/pulse/postprocess-swat-simulations-2-runoff-change-chonghua-yin/). For example, query table names, cloumns in a table.

To simplify reading, we put the code into a funtion. However, this time we only used the SUB table and the variable of water yield (WYLD, mm H2O). The net amount of water that leaves the subbasin and contributes to streamflow in the reach during the time step.

In [3]:
def read_sub(db_name):
    con = sqlite3.connect(db_name)
    cursor = con.cursor()

    df = pd.read_sql_query("SELECT SUB, YR, MO, WYLDmm from sub", con)
    df = df.set_index(['MO'])
    con.close()
    return df

In addition, we only care about seasonal changes. Therefore, have to convert monthly data into seasonal means.

In [4]:
def calculate_ssnmean(df):
    quarters = {1: 'DJF', 2: 'DJF', 3: 'MAM', 4: 'MAM', 5: 'MAM', 6: 'JJA',
            7: 'JJA', 8: 'JJA', 9: 'SON', 10: 'SON', 11: 'SON', 12: 'DJF'}
    
    ssndf = df.groupby(['SUB',quarters])['WYLDmm'].mean()
    ssndf = ssndf.reset_index()
    ssndf.set_index(['SUB'])
    ssndf = ssndf.rename(index=str, columns={"level_1":"SSN"})
    pivoted = ssndf.pivot(index='SUB', columns='SSN', values='WYLDmm')
    return pivoted

### 3.1 Read baseline water yields

In [5]:
db_name = 'data\\baseline\\result_664_monthly.db3'
df_bsl = read_sub(db_name)

### 3.2 Read water yield in future

In [6]:
db_name = 'data\\future\\result_664_monthly.db3'
df_fut = read_sub(db_name)

### 3.3 Calculate seasonal mean

In [7]:
pivoted_bsl = calculate_ssnmean(df_bsl)
pivoted_fut = calculate_ssnmean(df_fut)

### 3.4 Calculate seasonal changes

In [8]:
pivoted_ch = (pivoted_fut - pivoted_bsl)/pivoted_bsl*100.0
pivoted_ch = pivoted_ch.reindex()

## 4. Spatial join

Merge the geo-dataframe with the seasonal change dataframe. To merge two tables, have to rename the column of Subbasin to SUB in geo_df_ll

In [9]:
geo_df_ll.rename(columns = {"Subbasin" : "SUB"}, inplace = True)

Merge two table on the common column of SUB

In [10]:
geo_merge = geo_df_ll.merge(pivoted_ch, on='SUB', how='left')

## 5. Choropleth map

A [choropleth map](https://en.wikipedia.org/wiki/Choropleth_map) is a thematic map in which areas are shaded or patterned in proportion to the measurement of the statistical variable being displayed on the map, such as population density or per-capita income.

We use the changes in JJA to produce choropleth map as backgroud and then use popup text to show more information for each subbasin.

In [11]:
 def poptext(row):
        return "<h3> Summary of Water Yield Change (%)</h3>" \
            + "<p> The changes are calcualted as the relative changes between baseline and future periods.</p>"\
            + "Subbasin: {0:.0f}".format(row['SUB']) + "</br>" \
            + "Area: {0:.2f}".format(row['Area']) + " sq m.</br>" \
            + "DJF: {0:.2f}".format(row['DJF']) + "</br>" \
            + "MAM: {0:.2f}".format(row['MAM']) + "</br>" \
            + "JJA: {0:.2f}".format(row['JJA']) + "</br>" \
            + "SON: {0:.2f}".format(row['SON'])

In [12]:
map = folium.Map(location=[38, 72], zoom_start=7, tiles = "Stamen Terrain",)

map.choropleth(
    geo_data=geo_merge,
    data=geo_merge,
    columns=['SUB', 'JJA'],
    key_on='feature.properties.SUB',
    legend_name='Water Yield Change in JJA (%)', 
    fill_color='BuPu',
    fill_opacity=0.4,
    highlight=True)

for idx, row in geo_merge.iterrows():     
    txt = poptext(row)    
    iframe = folium.IFrame(html=txt, width=300, height=270)
    popup = folium.Popup(iframe, max_width=2650)
    folium.Marker([row['coords'][1], row['coords'][0]], 
                  popup=popup,
                  icon=folium.Icon(color='blue', icon='info-sign')).add_to(map)    
    
map

Save the map to html. Then you can dispaly the map in browser.

In [13]:
map.save('data\WYLD_Change_map.html')

## References

Fernando Pérez and Brian E. Granger. IPython: A System for Interactive Scientific Computing, Computing in Science & Engineering, 9, 21-29 (2007), DOI:10.1109/MCSE.2007.53

John D. Hunter. Matplotlib: A 2D Graphics Environment, Computing in Science & Engineering, 9, 90-95 (2007), DOI:10.1109/MCSE.2007.55

Wes McKinney. Data Structures for Statistical Computing in Python, Proceedings of the 9th Python in Science Conference, 51-56 (2010)

http://geopandas.org/

http://pysal.readthedocs.io/en/latest/

http://python-visualization.github.io/folium/

#![img](img\qqq.jpg)